<div id="colab_button">
  <h1>Whisper integration</h1>
  <a target="_blank" href="https://colab.research.google.com/github/mithril-security/blindai/blob/main/docs/docs/tutorials/api/whisper_tutorial.ipynb"> 
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</div>
______________________________

In this tutorial, we'll **transcribe an audio file** with **privacy guarantees** using the popular model **Whisper** with **BlindAI API**. Whisper is an open-source automatic speech recognition (ASR) system trained on 680,000 hours of multilingual and multitask supervised data collected from the web.

## Installation
________________________________________________________

Before we begin, you will need to install the [blindai](https://pypi.org/project/blindai/) Python library and download an example [test audio file](https://www2.cs.uic.edu/~i101/SoundFiles/taunt.wav). This one was provided by University of Illinois at Chicago  as part of their Computer Science 101 course.

In [ ]:
# install blindai
!pip install blindai

# Download our example audio file and save it as `taunt.wav`
!wget https://www2.cs.uic.edu/~i101/SoundFiles/taunt.wav -O taunt.wav

## Transcribing the audio file
______________________________________________

Speech-to-text operations are handled within the `Audio.transcribe` module of `BlindAI.api`.

To transcribe an audio file, we call the `transcribe` method and provide the path to our audio file as the `file` option. The Whisper model accepts a variety of audio file formats (m4a, mp3, mp4, mpeg, mpga, wav, webm).

In [2]:
import blindai

transcript = blindai.api.Audio.transcribe("taunt.wav")

>If you or your service provider are running your own instance of the BlindAI server and you don't wish to connect to our managed cloud solution, you can pass your `BlindAiConnection` client object to the `transcribe` method's `connection` option.

We can print out the returned `transcript` string variable to check our results are as expected.

In [3]:
print(transcript)

 Now go away, or I shall taunt you a second timer!


Our audio file has been correctly transcribed! 

Feel free to test this out with your own audio files 🔊


## Under the hood
______________________________________________

Let's take a moment to discuss what went on under the hood when we called the `blindai.api.Audio.transcribe()` method. This will allow you to understand better how data privacy is guaranteed by the hardware. 

### Connection

First, the `transcribe()` method connects to our managed cloud solution by calling the BlindAI.Core's `connect()` method, with the following code:

In [4]:
DEFAULT_BLINDAI_ADDR = "4.246.205.63"

connection = blindai.core.connect(
    DEFAULT_BLINDAI_ADDR,
    hazmat_http_on_unattested_port=True,
)

It is at this point that the attestation process is triggered. There is a technical explanation of what goes on in [our confidential computing guide](https://blindai.mithrilsecurity.io/en/latest/docs/getting-started/confidential_computing/). 

But if you haven't read it yet, all you need to know at this point is that the attestation process **verifies** that the **application code** running in the enclave has **not been modified**. To do so, it will check that the Mithril server instance is running a real enclave using **genuine hardware**. 

The client will also check the application code running in the enclave exactly matches the expected code for the latest release of BlindAI. The code is defined in the client's built-in `manifest.toml` file. This step prevents users from being able to connect with a version of the application which has been tampered with and may not be trustworthy.

We can quickly prove that this check is in place by doing the following:
1. Creating an empty `manifest.toml` file called `fake_manifest.toml`
2. Using the `connect()` method's `hazmat_manifest_path()` option to override the default path for the `manifest.toml` file with the path for our `fake_manifest.toml` file

Uncomment and run the following code to see this for yourself.

In [6]:
# !touch fake_manifest.toml

# connection = blindai.core.connect(
#             DEFAULT_BLINDAI_ADDR,
#             hazmat_http_on_unattested_port=True,
#             use_cloud_manifest=True,
#             hazmat_manifest_path="./fake_manifest.toml"
#         )

AttestationError: ignored

This causes the connection to be refused with an `AttestationError` because the application code cannot be verified.

> **Note**
>
> To test this more thoroughly, you can download our GitHub project and modify the server code stored in the `src` folder very slightly. 
>
>You then need to generate a new `manifest.toml` based on this modified source code by building the project. Once this is done, replace the path to the `fake_manifest.toml` in the previous example with a path to your newly generated `manifest.toml`. This will lead to the same error, since the server's hash received in the attestation report will not match the code in the newly generated `manifest.toml`.

### Querying the model

Once connected, the `transcribe()` method will prepare the audio file correctly for the model and call the BlindAI.Core `run_model()` method, passing it the ID for the default Whisper model and the now pre-processed input data:

```py
res = connection.run_model(model_hash=DEFAULT_WHISPER_MODEL, input_tensors=input_data)
```

Then, the method extracts the text transcription from the returned object and returns that to our BlindAI API user!




## Conclusions
____________________________

We have seen:
- How to **confidentially transcribe audio** using the Whisper model in BlindAI API.
- How **privacy** is guaranteed **under the hood**.

Please check out the rest of our BlindAI documentation to see more examples of how you can use BlindAI to query AI models without compromising the safety of user data or models.